In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *

### Data reading 

In [0]:
df = spark.read.format("parquet")\
          .load("abfss://bronze@databricksetee.dfs.core.windows.net/customers/")

df.display()

In [0]:
df = df.drop("_rescued_data")
#display(df)

In [0]:
df = df.withColumn("domain", split(col("email"), "@")[1])

df.display()


In [0]:
df1 = df.groupBy("domain").agg(count("customer_id").alias("total_customers")).sort(desc("total_customers"))
display(df1)


In [0]:
df_gmail = df.filter(col("domain") == "gmail.com")
#df_gmail.display()

df_yahoo = df.filter(col("domain") == "yahoo.com")
#df_yahoo.display()

df_hotmail = df.filter(col("domain") == "hotmail.com")
#df_hotmail.display()




In [0]:
df = df.withColumn("fullname", concat(col("first_name"), lit(" "), col("last_name")))
df = df.drop("first_name", "last_name")
df.display()


In [0]:
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("abfss://silver@databricksetee.dfs.core.windows.net/customers/")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@databricksetee.dfs.core.windows.net/customers/';


In [0]:
%sql
select * from databricks_cata.silver.customers_silver